In [ ]:
import shap
import sklearn

from sklearn.model_selection import train_test_split

from raiwidgets import ModelAnalysisDashboard
from responsibleai import ModelAnalysis

### Use adult census dataset

In [ ]:
X, y = shap.datasets.adult()
target_feature = 'income'
categorical_features = ['Workclass', 'Education-Num', 'Marital Status',
                        'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
continuous_features = ['Age', 'Capital Gain', 'Capital Loss', 'Hours per week']

y = [1 if x else 0 for x in y]

n_samples = 10000
X, y = sklearn.utils.resample(X, y, n_samples=n_samples, random_state=7, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

full_data = X.copy()
test_data = X_test.copy()
full_data[target_feature] = y
test_data[target_feature] = y_test

### Train a model

In [ ]:
model = sklearn.neighbors.KNeighborsClassifier()
model.fit(X_train, y_train)

### Compute Responsible AI model and dataset insights

In [ ]:
model_analysis = ModelAnalysis(model, full_data, test_data, target_feature, 'classification',
                               categorical_features=categorical_features)

# Queue Responsible AI insights
model_analysis.explainer.add()
model_analysis.counterfactual.add(continuous_features, 10, desired_class='opposite')
model_analysis.error_analysis.add()
model_analysis.causal.add()

# Compute insights
model_analysis.compute()

### Visualize insights in the model analysis dashboard

In [ ]:
ModelAnalysisDashboard(model_analysis)